# Cloudbutton geospatial use case: Model calculation

In [216]:
import os
import pathlib
import json
import subprocess
import shutil
import lithops
from lithops.storage import Storage

import numpy as np
import pdal
from osgeo import gdal
from scipy import ndimage

from posixpath import join as posix_join


In [217]:
DATA_BUCKET = 'cb-geospatial-wildfirev1'
COMPUTE_BACKEND = 'aws_lambda'
STORAGE_BACKEND = 'aws_s3'
STORAGE_PREFIX = 'aws_s3://'
INPUT_DATA_PREFIX = 'input-las-tiles/'

Experiment parameters


In [218]:
FCC_WINDOW = 3
FCC_BREAKPOINT = 0.01

---
Upload dataset
---


In [219]:
LOCAL_INPUT_DIR = 'input-las-tiles'

In [220]:
storage = Storage(backend=STORAGE_BACKEND)

2024-02-02 16:06:04,791 [INFO] aws_s3.py:68 -- S3 client created - Region: eu-west-1


In [221]:
bucket_objects = storage.list_keys(bucket=DATA_BUCKET)

for file_name in os.listdir(LOCAL_INPUT_DIR):
    if file_name not in bucket_objects:
        key = os.path.join(INPUT_DATA_PREFIX, file_name)
        with open(os.path.join(LOCAL_INPUT_DIR, file_name), 'rb') as file:
            print(f'Uploading {key}...')
            data = file.read()
            storage.put_object(bucket=DATA_BUCKET, key=key, body=data)
            print('Ok!')

Uploading input-las-tiles/lidarcatv01ls12f390612ed03.laz...
Ok!
Uploading input-las-tiles/lidarcatv01ls12f390614ed03.laz...
Ok!
Uploading input-las-tiles/lidarcatv01ls12f390616ed03.laz...
Ok!
Uploading input-las-tiles/lidarcatv01ls12f390618ed03.laz...
Ok!


---
Calculte DEM, DSM and CHM
---


In [222]:
storage.list_keys(bucket=DATA_BUCKET)

['input-las-tiles/lidarcatv01ls12f390612ed03.laz',
 'input-las-tiles/lidarcatv01ls12f390614ed03.laz',
 'input-las-tiles/lidarcatv01ls12f390616ed03.laz',
 'input-las-tiles/lidarcatv01ls12f390618ed03.laz']

In [223]:
def calculate_models(obj, storage):
    # Create temporary file paths
    tmp_path_prefix = '/tmp/geo/'
    if os.path.exists(tmp_path_prefix):
        shutil.rmtree(tmp_path_prefix)
    for subpath in ['dsm', 'dem', 'chm', 'aspect', 'slope', 'fcc']:
        os.makedirs(os.path.join(tmp_path_prefix, subpath), exist_ok=True)

    las_tile_filename = pathlib.Path(obj.key).name
    tile_key = pathlib.Path(obj.key).stem

    # Save obj to file
    data = obj.data_stream.read()
    input_file_path = os.path.join(tmp_path_prefix, las_tile_filename)
    with open(input_file_path, 'wb') as file:
        file.write(data)

    # DSM pipeline
    dsm_file_path = os.path.join(tmp_path_prefix, 'dsm', tile_key + '.gtiff')
    dsm_pipeline_json = {
        "pipeline": [
            {
                "type": "readers.las",
                "filename": f"{input_file_path}",
                "spatialreference": "EPSG:25830"
            },
            {
                "type": "filters.reprojection",
                "in_srs": "EPSG:25830",
                "out_srs": "EPSG:25830"
            },
            {
                "type": "filters.outlier",
                "method": "radius",
                "radius": 1.0,
                "min_k": 4
            },
            {
                "type": "filters.range",
                # Classification equals 2 (corresponding to noise points in LAS).
                "limits": "Classification![7:7]"
            },
            {
                "type": "filters.range",
                "limits": "returnnumber[1:1]"
            },
            {
                "type": "writers.gdal",
                "gdaldriver": "GTiff",
                "nodata": "-9999",
                "output_type": "max",
                "resolution": 1,
                "filename": f"{dsm_file_path}"
            }
        ]
    }
    dsm_pipeline_json_str = json.dumps(dsm_pipeline_json, indent=4)
    pipeline = pdal.Pipeline(dsm_pipeline_json_str)
    #pipeline.validate()
    #pipeline.loglevel = 8
    print('Executing DSM pipeline...')
    result = pipeline.execute()
    print(result)

    # DEM pipeline
    dem_file_path = os.path.join(tmp_path_prefix, 'dem', tile_key + '.gtiff')
    dem_pipeline_json = {
        "pipeline": [
            {
                "type": "readers.las",
                "filename": f"{input_file_path}",
                "spatialreference": "EPSG:25830"
            },
            {
                "type": "filters.reprojection",
                "in_srs": "EPSG:25830",
                "out_srs": "EPSG:25830"
            },
            {
                "type": "filters.assign",
                "assignment": "Classification[:]=0"
            },
            {
                "type": "filters.elm"
            },
            {
                "type": "filters.outlier",
                "method": "radius",
                "radius": 1.0,
                "min_k": 4
            },
            {

                "type": "filters.smrf",
                "ignore": "Classification[7:7]",
                "slope": 0.2,
                "window": 16,
                "threshold": 0.45,
                "scalar": 1.2
            },
            {
                "type": "filters.range",
                # Classification equals 2 (corresponding to ground in LAS).
                "limits": "Classification[2:2]",
            },
            {
                "type": "writers.gdal",
                "gdaldriver": "GTiff",
                "nodata": "-9999",
                "output_type": "max",
                "resolution": 1,
                "filename": f"{dem_file_path}"
            }
        ]
    }
    dem_pipeline_json_str = json.dumps(dem_pipeline_json, indent=4)
    pipeline = pdal.Pipeline(dem_pipeline_json_str)
    #pipeline.validate()  # Check if json options are good
    #pipeline.loglevel = 8
    print('Executing DEM pipeline...')
    result = pipeline.execute()
    print(result)

    # calculate CHM
    chm_file_path = os.path.join(tmp_path_prefix, 'chm', tile_key + '.tiff')
    cmd = ['gdal_calc.py', '-A', dem_file_path, '-B', dsm_file_path,
           '--calc="B-A"', '--NoDataValue=0', '--outfile', chm_file_path]
    p = subprocess.Popen(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE, close_fds=True)
    stdout, stderr = p.communicate()
    print(stdout, stderr)
    # assert p.returncode == 0

    # calculate aspect
    aspect_file_path = os.path.join(tmp_path_prefix, 'aspect', tile_key + '.tiff')
    cmd = ['gdaldem', 'aspect', dem_file_path, aspect_file_path, '-compute_edges']
    p = subprocess.Popen(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE, close_fds=True)
    stdout, stderr = p.communicate()
    print(stdout, stderr)
    # assert p.returncode == 0

    # calculate slope
    slope_file_path = os.path.join(tmp_path_prefix, 'slope', tile_key + '.tiff')
    cmd = ['gdaldem', 'slope', dem_file_path, slope_file_path, '-compute_edges']
    p = subprocess.Popen(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE, close_fds=True)
    stdout, stderr = p.communicate()
    print(stdout, stderr)
    # assert p.returncode == 0

    # calculate FCC
    in_ds = gdal.Open(dem_file_path)
    rows = in_ds.RasterYSize
    cols = in_ds.RasterXSize
    in_band = in_ds.GetRasterBand(1)
    data = in_band.ReadAsArray(0, 0, cols, rows).astype(np.float64)
    data[data > FCC_BREAKPOINT] = 1
    data[data <= FCC_BREAKPOINT] = 0

    # Computing fraction on the whole raster through a moving window.
    def _compute_fraction(array):
        nveg = np.sum(array == 1)
        total = len(array)
        out = (nveg/total)*100
        return(out)

    TCC = ndimage.generic_filter(data, _compute_fraction, size=FCC_WINDOW)

    gtiff_driver = gdal.GetDriverByName("GTiff")
    fcc_file_path = os.path.join(tmp_path_prefix, 'fcc', tile_key + '.tiff')
    out_ds = gtiff_driver.Create(fcc_file_path, cols, rows, 1, in_band.DataType)
    out_ds.SetProjection(in_ds.GetProjection())
    out_ds.SetGeoTransform(in_ds.GetGeoTransform())

    out_band = out_ds.GetRasterBand(1)
    out_band.WriteArray(TCC)
    # out_ds.BuildOverviews("Average", [2, 4, 8, 16, 32])
    out_ds.FlushCache()
    del in_ds, out_ds

    outputs = [dsm_file_path, dem_file_path, chm_file_path,
               aspect_file_path, slope_file_path, fcc_file_path]
    for output_path in outputs:
        if os.path.exists(output_path):
            with open(output_path, 'rb') as output_file:
                data = output_file.read()
                cos_key = output_path.replace(tmp_path_prefix, '')
                storage.put_object(bucket=DATA_BUCKET, key=cos_key, body=data)
        else:
            print(f'Failed to upload {output_path}')

    out = subprocess.check_output(['find', '/tmp/geo/'])
    return out

In [224]:
fexec = lithops.FunctionExecutor()

2024-02-02 16:06:32,202 [INFO] config.py:139 -- Lithops v3.1.0 - Python3.10
2024-02-02 16:06:32,210 [INFO] aws_s3.py:68 -- S3 client created - Region: eu-west-1
2024-02-02 16:06:33,971 [INFO] aws_lambda.py:106 -- AWS Lambda client created - Region: eu-west-1


In [225]:
os.path.join(STORAGE_PREFIX, DATA_BUCKET, INPUT_DATA_PREFIX)

'aws_s3://cb-geospatial-wildfirev1\\input-las-tiles/'

In [226]:
fs = fexec.map(calculate_models, posix_join(STORAGE_PREFIX, DATA_BUCKET, INPUT_DATA_PREFIX))

2024-02-02 16:06:34,040 [INFO] invokers.py:107 -- ExecutorID 4f70d2-18 | JobID M000 - Selected Runtime: monitoring5 - 3007MB
2024-02-02 16:06:34,574 [INFO] invokers.py:172 -- ExecutorID 4f70d2-18 | JobID M000 - Starting function invocation: calculate_models() - Total: 4 activations
2024-02-02 16:06:34,578 [INFO] invokers.py:208 -- ExecutorID 4f70d2-18 | JobID M000 - View execution logs at C:\Users\ubena\AppData\Local\Temp\lithops-root\logs\4f70d2-18-M000.log


In [227]:
res = fexec.get_result(fs=fs)

2024-02-02 16:06:34,608 [INFO] wait.py:98 -- ExecutorID 4f70d2-18 - Getting results from 4 function activations


    0%|          | 0/4  

2024-02-02 16:08:18,435 [INFO] executors.py:596 -- ExecutorID 4f70d2-18 - Cleaning temporary data


In [228]:
for r in res:
    print(r.decode('utf-8').strip())
    print('---')

/tmp/geo/
/tmp/geo/lidarcatv01ls12f390612ed03.laz
/tmp/geo/dem
/tmp/geo/dem/lidarcatv01ls12f390612ed03.gtiff
/tmp/geo/slope
/tmp/geo/slope/lidarcatv01ls12f390612ed03.tiff
/tmp/geo/fcc
/tmp/geo/fcc/lidarcatv01ls12f390612ed03.tiff
/tmp/geo/dsm
/tmp/geo/dsm/lidarcatv01ls12f390612ed03.gtiff
/tmp/geo/aspect
/tmp/geo/aspect/lidarcatv01ls12f390612ed03.tiff
/tmp/geo/chm
/tmp/geo/chm/lidarcatv01ls12f390612ed03.tiff
---
/tmp/geo/
/tmp/geo/dem
/tmp/geo/dem/lidarcatv01ls12f390614ed03.gtiff
/tmp/geo/slope
/tmp/geo/slope/lidarcatv01ls12f390614ed03.tiff
/tmp/geo/fcc
/tmp/geo/fcc/lidarcatv01ls12f390614ed03.tiff
/tmp/geo/dsm
/tmp/geo/dsm/lidarcatv01ls12f390614ed03.gtiff
/tmp/geo/lidarcatv01ls12f390614ed03.laz
/tmp/geo/aspect
/tmp/geo/aspect/lidarcatv01ls12f390614ed03.tiff
/tmp/geo/chm
/tmp/geo/chm/lidarcatv01ls12f390614ed03.tiff
---
/tmp/geo/
/tmp/geo/dem
/tmp/geo/dem/lidarcatv01ls12f390616ed03.gtiff
/tmp/geo/slope
/tmp/geo/slope/lidarcatv01ls12f390616ed03.tiff
/tmp/geo/fcc
/tmp/geo/fcc/lidarcatv01ls12